# SSWEI Class Example

This notebook demonstrates how to use the SSWEI class for calculating the Standardized Snow Water Equivalent Index (SSWEI) and classifying drought conditions. The SSWEI class provides an object-oriented interface to the SSWEI calculation and drought classification functionality.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Import snowdroughtindex package
from snowdroughtindex.core.sswei_class import SSWEI
from snowdroughtindex.core.dataset import SWEDataset

## 1. Loading SWE Data

First, let's load some SWE data using the SWEDataset class and then pass it to the SSWEI class.

In [ ]:
# Create a SWEDataset object
swe_dataset = SWEDataset()

# Load data from a file
data_file = '../data/swe_data.nc'  # Update with your actual data file path

# Check if the file exists
if os.path.exists(data_file):
    swe_dataset.load_from_file(data_file)
    print(f"Loaded data from {data_file}")
else:
    print(f"File {data_file} not found. Creating sample data...")
    
    # Create sample data for demonstration
    years = range(1980, 2021)  # 41 years of data
    months = range(1, 13)      # 12 months per year
    days = [1]                 # Just the first day of each month for simplicity
    
    # Create dates
    dates = []
    for year in years:
        for month in months:
            for day in days:
                try:
                    dates.append(datetime(year, month, day))
                except ValueError:
                    # Skip invalid dates (e.g., February 30)
                    pass
    
    # Create stations
    stations = [f'station_{i}' for i in range(1, 6)]
    
    # Create a DataFrame with random SWE values
    np.random.seed(42)  # For reproducibility
    data = {}
    for station in stations:
        # Create seasonal pattern with random noise and long-term trend
        values = []
        for i, date in enumerate(dates):
            # Seasonal pattern (higher in winter, lower in summer)
            month_factor = np.sin(2 * np.pi * (date.month - 1) / 12 - np.pi/2)
            seasonal = 100 * max(0, month_factor) + 10
            
            # Add some year-to-year variability (drought cycles)
            year_factor = np.sin(2 * np.pi * (date.year - 1980) / 20)  # 20-year cycle
            year_effect = 30 * year_factor
            
            # Add random noise
            noise = np.random.normal(0, 10)
            
            # Combine factors
            value = seasonal + year_effect + noise
            value = max(0, value)  # No negative SWE values
            values.append(value)
        
        data[station] = values
    
    df = pd.DataFrame(data, index=dates)
    
    # Create station information
    station_info = pd.DataFrame({
        'station_id': stations,
        'lat': np.random.uniform(40, 45, len(stations)),
        'lon': np.random.uniform(-120, -115, len(stations)),
        'elevation': np.random.uniform(1500, 3000, len(stations))
    })
    
    # Create SWEDataset with sample data
    swe_dataset = SWEDataset(df, station_info)
    print("Sample data created successfully.")

## 2. Creating an SSWEI Object

Now, let's create an SSWEI object using the SWE data.

In [ ]:
# Create an SSWEI object with the SWEDataset
sswei_obj = SSWEI(swe_dataset)
print(sswei_obj)

## 3. Calculating SSWEI

Let's calculate the SSWEI for the winter season (December to March).

In [ ]:
# Calculate SSWEI for the winter season (December to March)
try:
    sswei_obj.calculate_sswei(
        start_month=12,  # December
        end_month=3,     # March
        min_years=10,    # Minimum 10 years of data required
        distribution='gamma'  # Use gamma distribution
    )
    
    # Display the SSWEI results
    print("SSWEI Results:")
    display(sswei_obj.sswei_data.head())
    
    # Print summary statistics
    print("\nSSWEI Summary Statistics:")
    display(sswei_obj.sswei_data['SWEI'].describe())
    
    # Count drought classifications
    print("\nDrought Classification Counts:")
    display(sswei_obj.sswei_data['Drought_Classification'].value_counts())
except Exception as e:
    print(f"Error calculating SSWEI: {e}")
    print("Note: This may occur with the sample data due to insufficient seasonal data.")

## 4. Customizing Drought Classification Thresholds

Let's customize the thresholds used for drought classification.

In [ ]:
# Define custom thresholds
custom_thresholds = {
    "exceptional": -1.8,  # More severe threshold for exceptional drought
    "extreme": -1.3,      # More severe threshold for extreme drought
    "severe": -0.8,       # More severe threshold for severe drought
    "moderate": -0.4,     # Less severe threshold for moderate drought
}

# Apply custom thresholds and reclassify
try:
    sswei_obj.set_thresholds(custom_thresholds)
    sswei_obj.classify_drought()
    
    # Count drought classifications with custom thresholds
    print("Drought Classification Counts with Custom Thresholds:")
    display(sswei_obj.sswei_data['Drought_Classification'].value_counts())
except Exception as e:
    print(f"Error applying custom thresholds: {e}")

## 5. Visualizing SSWEI Time Series

Let's visualize the SSWEI time series with drought classification color coding.

In [ ]:
# Plot SSWEI time series
try:
    fig = sswei_obj.plot_sswei_timeseries()
    plt.show()
except Exception as e:
    print(f"Error plotting SSWEI time series: {e}")

## 6. Analyzing Drought Characteristics

Let's analyze drought characteristics such as duration, severity, and intensity.

In [ ]:
# Calculate drought characteristics
try:
    drought_chars = sswei_obj.calculate_drought_characteristics()
    
    if not drought_chars.empty:
        print("Drought Characteristics:")
        display(drought_chars)
        
        # Plot drought characteristics
        fig = sswei_obj.plot_drought_characteristics()
        plt.show()
    else:
        print("No drought events found in the data.")
except Exception as e:
    print(f"Error analyzing drought characteristics: {e}")

## 7. Analyzing Drought Trends

Let's analyze drought trends over time using a moving window approach.

In [ ]:
# Analyze drought trends with a 10-year moving window
try:
    trend_data = sswei_obj.analyze_drought_trends(window_size=10)
    
    if not trend_data.empty:
        print("Drought Trends:")
        display(trend_data.head())
        
        # Plot drought trends
        fig = sswei_obj.plot_drought_trends(window_size=10)
        plt.show()
    else:
        print("Not enough data for trend analysis.")
except Exception as e:
    print(f"Error analyzing drought trends: {e}")

## 8. Creating a Drought Classification Heatmap

Let's create a heatmap of drought classifications by decade.

In [ ]:
# Create a drought classification heatmap
try:
    fig = sswei_obj.plot_drought_classification_heatmap()
    plt.show()
except Exception as e:
    print(f"Error creating drought classification heatmap: {e}")

## 9. Analyzing Drought Severity Distribution

Let's analyze the distribution of drought severity values.

In [ ]:
# Plot drought severity distribution
try:
    fig = sswei_obj.plot_drought_severity_distribution()
    plt.show()
except Exception as e:
    print(f"Error plotting drought severity distribution: {e}")

## 10. Saving SSWEI Results

Finally, let's save the SSWEI results to a CSV file.

In [ ]:
# Save SSWEI results to a CSV file
try:
    output_file = '../data/processed/sswei_results.csv'
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Save the results
    sswei_obj.save_results(output_file)
except Exception as e:
    print(f"Error saving SSWEI results: {e}")

## 11. Summary

In this notebook, we've demonstrated how to use the SSWEI class for calculating the Standardized Snow Water Equivalent Index (SSWEI) and classifying drought conditions. We've covered:

1. Loading SWE data
2. Creating an SSWEI object
3. Calculating SSWEI
4. Customizing drought classification thresholds
5. Visualizing SSWEI time series
6. Analyzing drought characteristics
7. Analyzing drought trends
8. Creating a drought classification heatmap
9. Analyzing drought severity distribution
10. Saving SSWEI results

The SSWEI class provides a convenient object-oriented interface to the SSWEI calculation and drought classification functionality, making it easier to work with SSWEI data and analyze drought conditions.